# 1.0 Um problema de classificação de ponta-a-ponta usando NLP

## 1.1 Descrição do dataset

O dataset contém informações gerais de 5.000 processos julgados
nos Juizados Especiais Federais dentro das Seções Judiciárias do Tribunal Regional Federal da 5a Região (https://www.trf5.jus.br). Os dados são oriundos da raspagem da consulta pública processual. Além disso, ele possui 46 colunas, das quais duas possuem texto livre:
"conteudo_sentenca" e "conteudo_acordao".

O dataset pode ser baixado no link a seguir: https://jacob.al/dataset_juizados 

Ao longo dos notebooks, vão ser realizados os seguintes passos:

1. Importação do dataset **(concluído)**
2. Análise exploratória dos dados **(concluído)**
3. Pré-processamento **(concluído)**
4. Verificação dos dados
5. Segregação dos dados
6. Treinamento
7. Teste

<center><img width="600" src="https://drive.google.com/uc?export=view&id=1fKGuR5U5ECf7On6Zo1UWzAIWZrMmZnGc"></center>


## 1.2 Instalação e importação das bibliotecas e configuração do wandb

In [ ]:
!pip install wandb

In [ ]:
!pip install pytest pytest-sugar

In [ ]:
import wandb

In [ ]:
# Login no Weights & Biases
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## 1.2 Pytest


### 1.2.1 Criação e execução do arquivo de teste


In [ ]:
%%file test_data.py
import pytest
import wandb
import pandas as pd

# This is global so all tests are collected under the same run
run = wandb.init(project="nlp_bolsa", job_type="data_checks")

@pytest.fixture(scope="session")
def data():

    local_path = run.use_artifact("nlp_bolsa/preprocessed_data.csv:latest").file()
    df = pd.read_csv(local_path)

    return df

def test_data_length(data):
    """
    Nosso teste possui dados suficientes para continuar
    """
    assert len(data) > 1000


def test_number_of_columns(data):
    """
    Nós teste possui o número de colunas para continuar
    """
    assert data.shape[1] == 2

def test_column_presence_and_type(data):

    required_columns = {
        "assunto_cnj": pd.api.types.is_object_dtype,
        "conteudo_sentenca": pd.api.types.is_object_dtype
    }

    # Verifica a presença da coluna
    assert set(data.columns.values).issuperset(set(required_columns.keys()))

    for col_name, format_verification_funct in required_columns.items():

        assert format_verification_funct(data[col_name]), f"Column {col_name} failed test {format_verification_funct}"


def test_class_names(data):

    # Check that only the known classes are present
    known_classes = [
        "Direito Tributário",
        "Direito Civil",
        "Direito Previdenciário",
        "Direito Administrativo e outras matérias do Direito Público",
        "Direito do Consumidor",
        "Direito Processual Civil e do Trabalho",
        "Direito do Trabalho"
    ]

    assert data["assunto_cnj"].isin(known_classes).all()

Writing test_data.py


In [ ]:
!pytest . -vv

Test session starts (platform: linux, Python 3.7.13, pytest 3.6.4, pytest-sugar 0.9.5)
cachedir: .pytest_cache
rootdir: /content, inifile:
plugins: typeguard-2.7.1, sugar-0.9.5

 test_data.py::test_data_length ✓                                 25% ██▌       
 test_data.py::test_number_of_columns ✓                           50% █████     
 test_data.py::test_column_presence_and_type ✓                    75% ███████▌  
 test_data.py::test_class_names ✓                                100% ██████████

Results (6.18s):
       4 passed


In [ ]:
# Encerrando o run
run.finish()